# Compare densities obtained from different versions of the reference atlases

## Description

The mouse brain reference volumes are a pair of volumetric files from the Allen Institute for Brain Science (AIBS): the **Nissl** and the **annotations** volumes.

The **Nissl volume** shows every cell in the brain. It is used to obtain the cell distribution in the mouse brain.
The **annotations volume** provides a brain region id to each voxel of the Nissl volume and any volumetric dataset of the AIBS. The mouse brain regions are organized in a hierarchical tree, with leaves representing the finest subdivisions captured by the AIBS. This **hierarchy** is stored in a **json file** and is common to each reference atlas version. It is also available on the AIBS website.

These two whole brain volumes exist in multiple versions and need to be aligned to each other to produce the best results in terms of cell density estimation (see https://doi.org/10.1101/2021.11.20.469384 for more details).

To find the best pair of annotations volume and Nissl volume, we are plotting the neuron density estimates of the Cell Atlas obtained from each version against literature values.

In [2]:
import re
import numpy as np
import json
from voxcell import RegionMap, VoxelData
import matplotlib.pylab as plt
from matplotlib import patches, lines
from os.path import join

## Set matplotlib parameters

In [ ]:
SMALL_SIZE = 48
MEDIUM_SIZE = 48
BIGGER_SIZE = 48
plt.rcParams["font.family"] = "Arial"
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rcParams['xtick.major.pad']  = 9.0
plt.rcParams['axes.linewidth'] = 2.0
plt.rcParams['xtick.major.size' ] = 7.0*3.0
plt.rcParams['xtick.minor.size' ] = 4.0*3.0
plt.rcParams['ytick.major.size' ] = 7.0*3.0
plt.rcParams['ytick.minor.size' ] = 4.0*3.0
plt.rcParams['xtick.major.width'] = 2.4
plt.rcParams['xtick.minor.width'] = 1.8
plt.rcParams['ytick.major.width'] = 2.4
plt.rcParams['ytick.minor.width'] = 1.8

# color dictionary
CBcdict={
    'Bl':np.array([0,0,0]),
    'Or':np.array([.9,.6,0]),
    'SB':np.array([.35,.7,.9]),
    'bG':np.array([0,.6,.5]),
    'Ye':np.array([.95,.9,.25]),
    'Bu':np.array([0,.45,.7]),
    'Ve':np.array([.8,.4,0]),
    'rP':np.array([.8,.6,.7]),
}

In [ ]:
DATA_FOLDER = '/path/to/data/'
OUTPUT_FOLDER = "/path/to/output/"
ccfs = ["bbp", "ccfv2", "ccfv3"]  # subfolders containing the annotations and Nissl volumes
ccf_color_keys = ["Bu", "Or", "SB"]
ccf_fullname = ['CCFbbp', 'CCFv2', 'CCFv3']  # names to display on the figure
annotation_file = "annotations.nrrd"  # name given to each nrrd annotation file
region_hierarchy_file = "brain_regions.json"  # mouse brain region hierarchy file (json).
neuron_density_file = "neuron_density.nrrd"  # name given to each neuron density file
cell_density_file = "cell_density.nrrd"  # name given to each cell density file

In [ ]:
region_map = RegionMap.load_json(join(DATA_FOLDER, region_hierarchy_file)) # load json hierarchy file

## Extract neuron density literature values to compare the cell atlas results to

The following mean values and standard deviation were extracted from Lefort et al. 2016, Xu et al. 2010 and Gourfinkel-An et al. 2003.
More information on this extraction in Rodarie et al. 2021.

In [ ]:
# Layer 1 values
names_l1 = ["Prelimbic area", "Primary somatosensory area",
         "Primary visual area", "Primary somatosensory area, barrel field"]  # Names of the regions
abvs_l1 = ["PV", "SS", "VA", "BF"]  # Abbreviations to display
lit_refs_l1 = ["Xu et al. 2010", "Xu et al. 2010", "Xu et al. 2010", "Lefort et al. 2016"]  # Reference paper for each value
lit_means_l1 = np.array([11328, 18752, 19452, 2873.630917])  # Mean layer 1 value extracted from each paper
lit_stds_l1 = np.array([1648, 2456, 1868, 884.1941283])  # Standard deviation for each value

# Molecular layer neuron density value from Gourfinkel-An et al. 2003.
# We also display the granular layer densities for comparison.
# No values are available for these regions so the density value is set to 0 to not be displayed.
names_cbx = ["Lingula (I), granular layer", "Lingula (I), molecular layer",
         "Flocculus, granular layer", "Flocculus, molecular layer"]
abvs_cbx = ["Ling, GL", "Ling, ML", "Floc, GL", "Floc, ML"]
lit_means_cbx = np.array([0, 55100]*2)
lit_refs_cbx = np.array(['', 'Gourfinkel-An et al. 2003'] * 2)

## Bar plots of comparison between literature and generated cell atlas neuron density for each reference atlas pair

In [ ]:
# Find ids related to isocortex layer 1 for each region of literature.
ids_ctx = region_map.find("Isocortex", attr="name", with_descendants=True)
ids_regions_l1 = [list(ids_ctx & region_map.find("@.*" + name + ".*layer 1$",
                                              attr="name")) for name in names_l1]

In [ ]:
# Extract generated density values of the Cell Atlas for each L1 region
ccf_densities_l1 = np.zeros((len(ccfs), len(names_l1)), dtype=float)
for i, ccf in enumerate(ccfs):
    annotation = VoxelData.load_nrrd(join(DATA_FOLDER, ccf, annotation_file)).raw
    neuron_density = VoxelData.load_nrrd(join(DATA_FOLDER, ccf, neuron_density_file)).raw
    for j, id_ in enumerate(ids_regions_l1):
        ccf_densities_l1[i, j] = np.mean(neuron_density[np.isin(annotation, id_)])

In [ ]:
fig = plt.figure(figsize=(15,10))
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
width = 1.0 / (len(ccfs) + 1) - 0.02
min_max = np.vstack((lit_stds_l1, lit_stds_l1))

for i, ccf in enumerate(ccfs):
    x = np.arange(len(names_l1)) * (width + 0.02) * (len(ccfs) + 1) + (width * i + width / 2)
    plt.barh(x, np.array(ccf_densities_l1[i]), height=width, color = CBcdict[ccf_color_keys[i]])
x = np.arange(len(names_l1)) * (width + 0.02) * (len(ccfs) + 1) + (width * len(ccfs) + width / 2)
bar4 = plt.barh(x, np.array(lit_means_l1),
                height=width, color = CBcdict["Ve"], xerr=min_max, edgecolor='black', linewidth=5)
for ibar, bar in enumerate(bar4):
    plt.annotate(lit_refs_l1[ibar],xy=(bar.get_width(), bar.get_y() + bar.get_height() / 2),
                 xytext=(50, 0),
                 textcoords="offset points",
                 ha='left', va='center')

x = np.arange(len(ccfs) + 1) + 0.5
plt.yticks(x, abvs_l1)
plt.ylim([0, len(x)])
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().yaxis.set_ticks_position('left')
plt.gca().xaxis.set_ticks_position('bottom')
plt.xlabel("Neuron densities [$mm^{-3}$]")
plt.tight_layout()
plt.savefig(join(OUTPUT_FOLDER, 'L1_dens_comparison.png'), dpi=200, facecolor="white")

In [ ]:
ids_regions_cbx = [list(region_map.find("@.*" + re.escape(name) + "$", attr="name")) for name in names_cbx]

In [ ]:
# Extract generated density values of the Cell Atlas for each cerebellar region
# The ccfv3 does not have the granular molecular subdivisions so it is not compared to the others.
ccf_densities_cbx = np.zeros((len(ccfs), len(names_cbx)), dtype=float)
ccf_cbx = ccfs[:-1]
for i, ccf in enumerate(ccf_cbx):
    annotation = VoxelData.load_nrrd(join(DATA_FOLDER, ccf, annotation_file)).raw
    neuron_density = VoxelData.load_nrrd(join(DATA_FOLDER, ccf, neuron_density_file)).raw
    for j, id_ in enumerate(ids_regions_cbx):
        ccf_densities_cbx[i, j] = np.mean(neuron_density[np.isin(annotation, id_)])

Densities of granular layer are an order of magnitude greater than the molecular layer
This makes it difficult to see the differences between the ccfs versions
To counter this problem, we are normalizing according to the largest value for each region.

In [ ]:
fig = plt.figure(figsize=(15,10))
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
width = 1.0 / (len(ccf_cbx) + 1) - 0.02

max_cbx = np.max(ccf_densities_cbx, axis=0)
for i, ccf in enumerate(ccf_cbx):
    x = np.arange(len(names_cbx)) * (width + 0.02) * (len(ccf_cbx) + 1) + (width * i + width / 2)
    plt.barh(x, np.array(ccf_densities_cbx[i]) / max_cbx, height=width, color = CBcdict[ccf_color_keys[i]])
x = np.arange(len(names_cbx)) * (width + 0.02) * (len(ccf_cbx) + 1) + (width * len(ccf_cbx) + width / 2)
bar4 = plt.barh(x, np.array(lit_means_cbx) / max_cbx,
                height=width, color = CBcdict["Ve"], edgecolor='black', linewidth=5)
for ibar, bar in enumerate(bar4):
    plt.annotate(lit_refs_cbx[ibar],xy=(bar.get_width(), bar.get_y() + bar.get_height() / 2),
                 xytext=(50, 0),
                 textcoords="offset points",
                 ha='left', va='center')

plt.yticks((x.astype(float)), abvs_cbx)
plt.ylim([0, len(x)])
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().yaxis.set_ticks_position('left')
plt.gca().xaxis.set_ticks_position('bottom')
plt.xlabel("Neuron densities [$mm^{-3}$]")
plt.tight_layout()
plt.savefig(join(OUTPUT_FOLDER, 'cereb_dens_comparison.png'), dpi=200, facecolor="white")

In [ ]:
# Plot the legend for previous figures in a separate figure.
plt.figure(figsize=(7, 5))
patchs = [patches.Patch(color=color) for color in [CBcdict["Ve"]] + [CBcdict[k] for k in ccf_color_keys]]
patchs[0].set_linewidth(3)
patchs[0].set_ec('black')
plt.axis("off")
plt.legend(patchs, ['Literature'] + ccf_fullname, loc='center')
plt.tight_layout()
plt.savefig(join(OUTPUT_FOLDER, 'legend_ann.png'), dpi=200, facecolor="white")

## Plot generated all densities against literature

We use here the literature values extracted from the review of Keller et al. 2018 (10.3389/fnana.2018.00083)
The cell and neuron values from the studies are stored in the provided "keller_review.json"

In [4]:
lit_dict_keller = json.load(open("../../atlas_densities/app/data/measurements/keller_review_2018.json", "r"))

In [ ]:
def hex_to_rgb(value):
    """
    Converts a Hexadecimal color into its RGB value counterpart.

    Args:
        value: string hexadecimal color to convert.
    Returns:
        List of the Red, Green, and Blue components of the color
    """

    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[c:c + lv // 3], 16) for c in range(0, lv, lv // 3))

The loading of all the densities values takes some time and a lot of memory space. It is therefore adviced to run each ccf plot separately.

In [ ]:
ccf = ccfs[0]
cellCategories = ["Cells", "Neurons"]
print("Loading generated densities for: " + ccf)
annotation = VoxelData.load_nrrd(join(DATA_FOLDER, ccf, annotation_file)).raw
neuron_density = VoxelData.load_nrrd(join(DATA_FOLDER, ccf, neuron_density_file)).raw
cell_density = VoxelData.load_nrrd(join(DATA_FOLDER, ccf, neuron_density_file)).raw
litArrayCCF = []
genArrayCCF = []
clrArrayCCF = []
categoryCCF = []
for icategory, (category_name, density_file) in enumerate(zip(cellCategories, [cell_density, neuron_density])):
    print("Loading " + category_name)
    litDicts = lit_dict_keller[category_name]
    litArray = []
    genArray = []
    clrArray = []
    for region_name, (values, ref) in litDicts.items():
        ids_regions = list(region_map.find(region_name, attr="name", with_descendants=True))
        if ids_regions:
            filter_region = np.isin(annotation, ids_regions)
            if filter_region.any():
                genArray.append(np.nanmean(density_file[filter_region]))
                litArray.append(values)
                clrArray.append(hex_to_rgb(region_map.get(ids_regions[0], "color_hex_triplet")))
    litArrayCCF += litArray
    genArrayCCF += genArray
    clrArrayCCF += clrArray
    categoryCCF += [icategory] * len(litArray)
clrArrayCCF = np.array(clrArrayCCF) / 255

In [ ]:
markers = ['o','v']
figScale = 1.8
oriFS =  7.0
ms_ = 14.0
lims = np.array([1e4, 2323608.559230513])

fig, ax = plt.subplots(figsize=(10, 10))
ax.tick_params(top=True, labeltop=True, right=True, labelright=True)
ax.loglog(lims, lims, "-", color = 'black')
ax.plot(lims * 4.1486, lims, "--", color='black')
ax.plot(lims / 4.1486, lims, "--", color ='black')
for il in range(len(litArrayCCF)):
    ax.loglog(len(litArrayCCF[il]) * [genArrayCCF[il]], litArrayCCF[il], marker=markers[categoryCCF[il]], c=clrArrayCCF[il], markersize=ms_)
    if len(litArrayCCF[il])>1:
        ax.loglog(len(litArrayCCF[il])*[genArrayCCF[il]], litArrayCCF[il], color=clrArrayCCF[il], linewidth=ms_ * 0.25 )
ax.set_ylabel(r"Literature density [$mm^{-3}$]")
ax.set_xlabel(r"Generated density [$mm^{-3}$]")
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.tick_params(axis='x', which='major', pad=20)
ax.tick_params(which='both', direction='in', labeltop=False, labelright=False)
plt.legend([lines.Line2D([0], [0], linestyle='none', mfc='black', color='black', marker=item) for item in np.array(markers)], np.array(cellCategories),
       numpoints=1, markerscale=2.0, loc="upper left", borderaxespad=0.0, fontsize=30)
plt.tight_layout(pad=0)
plt.savefig(join(OUTPUT_FOLDER, "cell_distribution", "Lit_Gen_"+ccf+".png"), dpi=200, facecolor="white")

## Plot histogram of generated densities

In [ ]:
all_regions_id = region_map.find("Basic cell groups and regions", attr="name", with_descendants=True)
genArray = []
clrArray = []
print("Loading all neuron densities for " + ccf)
for id_ in all_regions_id:
    ids_regions = list(region_map.find(id_, attr="id", with_descendants=True))
    filter_region = np.isin(annotation, ids_regions)
    if filter_region.any():
        genArray.append(np.nanmean(neuron_density[filter_region]))
        clrArray.append(hex_to_rgb(region_map.get(id_, "color_hex_triplet")))
clrArray = np.array(clrArray) / 255

In [ ]:
epsilon = 1e-5
nb_bins = 150
ff,ax=plt.subplots(figsize=(10, 10))
binsTMP = np.logspace(np.log10(1.0), np.log10(np.max(np.array(genArray)) + epsilon), nb_bins)
_, bins_, _ = plt.hist(genArray, bins = binsTMP, edgecolor='black', linewidth=1.2)
bin_densities = [[] for _ in range(nb_bins - 1)]

for i, density in enumerate(genArray):
    binID = np.where(density <= np.array(bins_))[0][0] - 1  # First bin greater than density
    bin_densities[binID].append(i)

for i, bin_ in enumerate(bin_densities):
    tmpArrayForSorting = []
    for index_dens in bin_:
        # Sort regions by color green on top, yellow at bottom
        tmpArrayForSorting.append(-(clrArray[index_dens][0] * 255 * 255 + clrArray[index_dens][1] * 255 + clrArray[index_dens][2]))
    bin_values_sorted = np.array(bin_)[np.argsort(np.array(tmpArrayForSorting))]

    for iaaV, aaV in enumerate(bin_values_sorted):
        ax.add_patch(patches.Rectangle((bins_[i], iaaV),
                                        width=bins_[i+1] - bins_[i], height=1.0,
                                        color=np.array(clrArray[aaV])))
res = plt.hist(genArray, bins=binsTMP, fill=False,
               histtype='step', color = "black", linewidth=2.0)
plt.gca().set_xscale("log")
plt.xlabel("Neuron density [$mm^{-3}$]")
plt.ylabel("Number of regions")
plt.xlim(lims)
plt.ylim([0, 65])
plt.tick_params(which='both', direction='in', labeltop=False, labelright=False)

ax.tick_params(axis='x', which='major', pad=20)
plt.tight_layout(pad=0)
plt.savefig(join(OUTPUT_FOLDER, "histNeuronDens_"+ ccf + ".png"),
            dpi=200, facecolor="white")